In [7]:
from google.colab import drive
drive.mount('/content/drive')
CSV_PATH = "/content/drive/My Drive/Completed_CSV/react_eval_20250803_191703.csv"

Mounted at /content/drive


In [8]:
!pip -q install evaluate bert_score rouge-score > /dev/null


In [9]:
import re, numpy as np, pandas as pd, evaluate, bert_score

df = pd.read_csv(CSV_PATH).fillna("")

rouge  = evaluate.load("rouge")

def ema(preds, refs):
    return np.mean([p == r for p, r in zip(preds, refs)])

def rouge_l(preds, refs):
    return rouge.compute(predictions=preds, references=refs)["rougeL"]

def _to_set(txt):
    return {t.strip() for t in re.split(r"[;,]", txt.lower()) if t.strip()}

def set_f1(preds, refs):
    scores = []
    for p, r in zip(preds, refs):
        P, R = _to_set(p), _to_set(r)
        if not P and not R: scores.append(1); continue
        if not P or not R:  scores.append(0); continue
        inter = len(P & R)
        prec  = inter / len(P);  rec = inter / len(R)
        scores.append(0 if prec+rec == 0 else 2*prec*rec/(prec+rec))
    return np.mean(scores)

BLACK = {"oven","bowl","pan","tray","sheet","mixer",
         "fork","knife","spoon","plate","°f","°c",
         "minutes","minute","timer","preheat"}

def non_food(preds):
    return np.mean([
        1 if any(tok in BLACK for tok in re.findall(r"\w+", p.lower())) else 0
        for p in preds
    ])

def bert_score_f1(preds, refs):
    preds = [p or "empty" for p in preds]
    refs  = [r or "empty" for r in refs]
    return bert_score.score(preds, refs, lang="en", verbose=False)[2].mean().item()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
preds = df["react_answer"]
refs  = df["ground_truth"]

metrics = {
    "EMA"          : ema(preds, refs),
    "RougeL"       : rouge_l(preds, refs),
    "SetF1"        : set_f1(preds, refs),
    "NonFood"      : non_food(preds),
    "BERTScore_F1" : bert_score_f1(preds, refs),
}

metrics["StrictScore"] = (
      0.70 * metrics["BERTScore_F1"]
    + 0.15 * metrics["SetF1"]
    + 0.10 * metrics["RougeL"]
    - 0.05 * metrics["NonFood"]
)

print("\n====== Evaluation Summary ======")
for k, v in metrics.items():
    print(f"{k:<12}: {v:.3f}")



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



====== Evaluation Summary ======
EMA         : 0.108
RougeL      : 0.155
SetF1       : 0.108
NonFood     : 0.000
BERTScore_F1: 0.849
StrictScore : 0.626


In [15]:
# 💾 Save metrics to Google Drive /Evaluation_ReAct/metrics_*.json
import json, datetime, pathlib, IPython.display as dsp
from google.colab import drive

# 1️⃣ Mount Google Drive (prompts once per session)
drive.mount('/content/drive')

# 2️⃣ Ensure target folder exists
gdrive_dir = pathlib.Path("/content/drive/My Drive/Evaluation_ReAct")
gdrive_dir.mkdir(parents=True, exist_ok=True)

# 3️⃣ Save with timestamped filename
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_path  = gdrive_dir / f"metrics_{timestamp}.json"

with open(out_path, "w") as f:
    json.dump(metrics, f, indent=2)

print("Metrics JSON saved →", out_path)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Metrics JSON saved → /content/drive/My Drive/Evaluation_ReAct/metrics_20250804_150904.json
